In [1]:
import cx_Oracle
import pandas as pd
import sys

In [30]:
ip_uat = '172.18.20.70:/FSCLIVE'
ip_live = '172.18.20.140:/FSCLIVE'
def selectfunc( ip_for = ip_uat, \
               verbose = 0, \
               entity = ['FSCGKK','FSCGKL','FSCGMH','FSCGPG','FSCPHA','FSCPHAK','FSCPHBP','FSCPHC','FSCPHI','FSCPHK','FSCPHKL',\
          'FSCPHM','FSCPHP','FSCPHSP'] \
              ):
    df = pd.DataFrame()
    for i in range(0,len(entity)):
        con = cx_Oracle.connect(entity[i], '<put password here>', ip_uat)
        if verbose == 1:
            print 'Connection successful to ', entity[i], ' '
        cur = con.cursor()
        #cur.execute('select * from gnuser')
        #cur.execute("select to_char(discharge_datetime,'rrrr-mm') from fdepisode fe") 
        cur.execute("select gl.acc_period, gga.GLACCOUNT_DESCRIPTION, sum(decode(dc_marker,'D',amount,-amount)) AMOUNT from gl_post_full gl, gnglaccount gga, journalentry je, boepisodeaccounttxn beat, gnchargeitem gci where gl.account_code = gga.glaccount_code and to_number(gl.filler2) = je.journalentry_id and je.BOEPISODEACCOUNTTXN_ID = beat.BOEPISODEACCOUNTTXN_ID  and beat.GNTXNCODE_ID = gci.GNTXNCODE_ID and gci.CHARGE_GROUP <> '2' and journal_type = 'HISR' and account_code like '1406%' and acc_period >= '2016001' group by gl.acc_period, gga.GLACCOUNT_DESCRIPTION")
        curdescription = pd.Series(cur.description)
        df_temp = pd.DataFrame(cur.fetchall(),columns=curdescription.apply(lambda x: x[0]))
        df_temp['Entity'] = entity[i][3:]
        df = df.append(df_temp)

    df = df[df.columns.tolist()[-1:] + df.columns.tolist()[:-1]]
    cur.close()
    con.close()
    return df

In [31]:
### Usage 
## df = selectfunc(ip, verbose)

df = selectfunc(ip_live,1)
df.head()

Connection successful to  FSCGKK  
Connection successful to  FSCGKL  
Connection successful to  FSCGMH  
Connection successful to  FSCGPG  
Connection successful to  FSCPHA  
Connection successful to  FSCPHAK  
Connection successful to  FSCPHBP  
Connection successful to  FSCPHC  
Connection successful to  FSCPHI  


OperationalError: ORA-03113: end-of-file on communication channel
Process ID: 45932
Session ID: 2207 Serial number: 5579


In [25]:
df.to_csv('C:\OneDrive - Parkway Pantai\WoRk\DASHBOARDS\Revenue by Fin-call and by month\outputfromquery.csv')

In [28]:
df.groupby(by='Entity').count()

Entity
GKK      5562
GKL     30459
GMH      4159
GPG     35768
PHA     10372
PHAK    28640
PHBP    16935
PHC     18144
PHI     26836
PHK     13233
PHKL    38453
PHM      4342
PHP     17287
PHSP    11700
Name: DISCHARGE_MONTH, dtype: int64